### 작업형 제1유형

#### 1.

In [3]:
import pandas as pd

lst = [2, 3, 3.2, 5, 7.5, 10, 11.8, 12, 23, 25, 31.5, 34]
q1, q3 = pd.Series(lst).quantile([0.25, 0.75])

print(int(abs(q1 - q3)))

18


#### 2.

In [13]:
import pandas as pd

df = pd.read_csv('../yemoonsaBigdata/datasets/Part3/402_facebook.csv')

# print(df.columns) # num_loves, num_wows

df['tmp'] = (df['num_loves'] + df['num_wows']) / df['num_reactions']

tmp = df.loc[(df['tmp'] > 0.4) & (df['tmp'] < 0.5), :]
print(len(tmp[tmp['status_type'] == 'video']))

# print(df['num_loves_wows'])

90


#### 3.

In [28]:
import pandas as pd


df = pd.read_csv('../yemoonsaBigdata/datasets/Part3/403_netflix.csv')
new_df = df.loc[df['date_added'].str.contains("January") & df['date_added'].str.contains("2018"), :]

print(len(new_df.loc[new_df['country'] == 'United Kingdom']))

6


<br><br><br>

### 작업형 제2유형

- 분류 문제 (multi label)
- macro f1 score

In [18]:
import pandas as pd
import numpy as np

X_train = pd.read_csv('../yemoonsaBigdata/datasets/Part3/404_x_train.csv')
y_train = pd.read_csv('../yemoonsaBigdata/datasets/Part3/404_y_train.csv')

X_test = pd.read_csv('../yemoonsaBigdata/datasets/Part3/404_x_test.csv')

unused_cols = ['ID']
num_cols = ['Age', 'Work_Experience', 'Family_Size']
cat_cols = ['Gender', 'Ever_Married', 'Graduated', 'Profession', 'Spending_Score']
y_cols = ['Segmentation']

# 범주형 데이터 전처리
from sklearn.preprocessing import LabelEncoder

X = pd.concat([X_train[cat_cols], X_test[cat_cols]])

for col in cat_cols:
    label_encoder = LabelEncoder()
    label_encoder.fit(X[col])
    
    X_train[col] = label_encoder.transform(X_train[col])
    X_test[col] = label_encoder.transform(X_test[col])
    
# 정답 데이터 전처리
for col in y_cols:
    y_label_encoder = LabelEncoder()
    y_label_encoder.fit(y_train[col])
    
    y_train[col] = y_label_encoder.transform(y_train[col])


# 학습, 검증 데이터셋 분리
from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(X_train[num_cols+cat_cols], y_train[y_cols], test_size=0.3, stratify=y_train[y_cols])


# 수치형 데이터 전처리
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_tr[num_cols])

X_tr[num_cols] = scaler.transform(X_tr[num_cols])
X_val[num_cols] = scaler.transform(X_val[num_cols])
X_test[num_cols] = scaler.transform(X_test[num_cols])


# 학습
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import f1_score

'''
model_rf = RandomForestClassifier()
model_rf.fit(X_tr, y_tr.values.squeeze())

pred_val = model_rf.predict(X_val)
print(f1_score(y_val, pred_val, average='macro'))

model_xgb = XGBClassifier()
model_xgb.fit(X_tr, y_tr.values.squeeze())

pred_val = model_xgb.predict(X_val)
print(f1_score(y_val, pred_val, average='macro'))
'''    

model_xgb1 = XGBClassifier(n_estimators=50, max_depth=5)
model_xgb1.fit(X_tr, y_tr.values.squeeze())

pred_val = model_xgb1.predict(X_val)
print(f1_score(y_val, pred_val, average='macro'))


# 테스트
pred = model_xgb1.predict(X_test[num_cols+cat_cols])
pred = y_label_encoder.inverse_transform(pred)

result = pd.DataFrame({
    'ID': X_test['ID'],
    'Segmentation': pred
})

result.to_csv('../yemoonsaBigdata/res/2.csv', index=False)

0.5082117976695973
